### **Cell 1: Setup, Geometry, and Vectorized Definitions**

**Mathematical Description:**
This cell establishes the algebraic geometry framework.
1.  **Hyperelliptic Curve:** Defined by $y^2 = \prod_{i=1}^{2g+2} (x - a_i)$. The branch points $a_i$ are generated on a perturbed circle.
2.  **Holomorphic Differentials:** $\omega_k = \frac{z^k}{\sqrt{P(z)}} dz$.
    *   *Optimization:* For $g=30$, calculating $P(z)$ directly overflows floating-point numbers. We compute in log-space: $\omega_k = \exp(k \ln z - \frac{1}{2}\sum \ln(z-a_i))$.
3.  **Vectorized Integration:** Instead of `mp.quad` (scalar CPU), we define `integrate_row_torch`. This computes the path integral $\int_{P_0}^z \omega$ for an entire row of the grid ($W$ points) simultaneously using a discretized path with $N$ steps.


In [2]:
# @title Setup: install, imports, mount Drive, shared config, genus-30 cuts
!pip install -q torch numpy mpmath tqdm

import os, time, math, json, numpy as np, torch, mpmath as mp
from tqdm import tqdm
# from google.colab import drive

# -----------------------------
# 1. Device Configuration
# -----------------------------
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    DTYPE = torch.complex128 # CUDA supports double precision
    print("✅ Using Device: CUDA (High Precision)")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    DTYPE = torch.complex64 # MPS is optimized for Float32/Complex64
    print("✅ Using Device: MPS (Standard Precision - LogSpace math used for stability)")
else:
    DEVICE = torch.device("cpu")
    DTYPE = torch.complex128
    print("⚠️ Using Device: CPU")

# -----------------------------
# Drive & save directory
# -----------------------------
DRIVE_FOLDER = "AJ_Tables_g30_cutaware"
# drive.mount('/content/drive', force_remount=True)
SAVE_DIR = f"./{DRIVE_FOLDER}"
os.makedirs(SAVE_DIR, exist_ok=True)

# File paths
OMEGAS_PATH    = os.path.join(SAVE_DIR, "aj_omegas_genus30_cutaware.pt")
INTEGRALS_PATH = os.path.join(SAVE_DIR, "aj_integrals_genus30_cutaware.pt")

# -----------------------------
# Genus & grid
# -----------------------------
genus = 30
H = W = 96
r_min, r_max = -6.0, 6.0
i_min, i_max = -6.0, 6.0
grid_r = np.linspace(r_min, r_max, W).astype(np.float64)
grid_i = np.linspace(i_min, i_max, H).astype(np.float64)

mp.mp.dps = 50
base_point_complex = complex(r_min - 2.0, i_min - 2.0)

# -----------------------------
# Helper: atomic save
# -----------------------------
def atomic_torch_save(obj, path):
    tmp = path + ".tmp"
    torch.save(obj, tmp)
    os.replace(tmp, path)

# -----------------------------
# Genus-30 cuts (CPU Logic - Keep as is for geometry generation)
# -----------------------------
def make_hyperelliptic_cuts(g, radius=4.0, jitter=0.25, seed=123):
    rng = np.random.RandomState(seed)
    m = 2*g + 2
    thetas = np.linspace(0, 2*np.pi, m, endpoint=False)
    rng.shuffle(thetas)
    radii = radius * (1.0 + jitter * (rng.rand(m) - 0.5))
    pts = radii * np.exp(1j * thetas)
    
    scale = max((pts.real.max()-pts.real.min())/(r_max-r_min+1e-6),
                (pts.imag.max()-pts.imag.min())/(i_max-i_min+1e-6))
    if scale > 0.85:
        pts = pts / (scale/0.85)

    remaining = list(range(m))
    cuts = []
    while remaining:
        i = remaining.pop(0)
        pi = pts[i]
        dists = [(j, abs(pi - pts[j])) for j in remaining]
        j = min(dists, key=lambda t: t[1])[0]
        remaining.remove(j)
        a, b = pi, pts[j]
        mid = 0.5*(a+b)
        a = a + 0.05*(a - mid)
        b = b + 0.05*(b - mid)
        cuts.append((complex(a), complex(b)))
    
    if len(cuts) > g+1:
        cuts.sort(key=lambda ab: -abs(ab[0]-ab[1]))
        cuts = cuts[:g+1]
    return cuts

branch_cuts = make_hyperelliptic_cuts(genus)
branch_pts_list = [a for ab in branch_cuts for a in ab]
print(f"genus={genus} → cuts={len(branch_cuts)} ; total branch points={len(branch_pts_list)}")

# -----------------------------
# GPU-Accelerated Math Definitions
# -----------------------------

# Move branch points to GPU once
# Shape: (1, 1, Num_Branch_Pts) for broadcasting
branch_pts_tensor = torch.tensor(branch_pts_list, device=DEVICE, dtype=DTYPE).reshape(1, 1, -1)

def omega_torch(k, z_tensor):
    """
    Vectorized ω_k(z) calculation using Log-Sum-Exp for numerical stability.
    z_tensor: (Steps, Width) complex tensor
    """
    # 1. Compute (z - a_i) for all i
    # Shape: (Steps, Width, Num_Branch_Pts)
    diffs = z_tensor.unsqueeze(-1) - branch_pts_tensor
    
    # 2. Compute log denominator: 0.5 * sum(log(z - a_i))
    # Complex log: log(r) + i*theta
    log_diffs = torch.log(diffs)
    log_denom = 0.5 * torch.sum(log_diffs, dim=-1)
    
    # 3. Compute log numerator: k * log(z)
    log_num = k * torch.log(z_tensor)
    
    # 4. Combine
    return torch.exp(log_num - log_denom)

def integrate_row_torch(k, z_row, base_pt, steps=1000):
    """
    Computes the path integral from base_pt to every point in z_row (a 1D tensor).
    Uses a straight line path discretized into `steps`.
    """
    # z_row shape: (Width,)
    W_dim = z_row.shape[0]
    
    # 1. Create Parametric Path t in [0, 1]
    # Shape: (Steps, 1)
    real_dtype = DTYPE.real_dtype if hasattr(DTYPE, 'real_dtype') else torch.float32 # Fix for older torch
    if DTYPE == torch.complex128: real_dtype = torch.float64
    
    t = torch.linspace(0, 1, steps, device=DEVICE, dtype=real_dtype).unsqueeze(1)
    
    # 2. Broadcast path
    # path(t) = base + t * (target - base)
    # Shape: (Steps, Width)
    diff = z_row - base_pt
    path = base_pt + t.type(DTYPE) * diff.unsqueeze(0)
    
    # 3. Evaluate Differential
    vals = omega_torch(k, path)
    
    # 4. Integrate (Trapezoidal / Riemann Sum)
    # int = sum(vals) * dt
    # dt = (z - base) / steps
    dt = diff / steps
    
    # Sum along time dimension (dim 0)
    integral = torch.sum(vals, dim=0) * dt
    
    return integral

# -----------------------------
# Metadata
# -----------------------------
COMMON_META = {
    "genus": genus,
    "branch_cuts": branch_cuts,
    "branch_pts": np.array(branch_pts_list, dtype=np.complex128),
    "grid_r": grid_r, "grid_i": grid_i,
    "meta": {"dtype": str(DTYPE), "base_point": base_point_complex,
             "grid_shape": (H, W), "ranges": (r_min, r_max, i_min, i_max)}
}

def ensure_config_compatible(pkg):
    assert int(pkg["genus"]) == genus
    assert pkg["meta"]["grid_shape"] == (H, W)
    assert np.allclose(pkg["grid_r"], grid_r)

✅ Using Device: MPS (Standard Precision - LogSpace math used for stability)
genus=30 → cuts=31 ; total branch points=62


This is a utility cell. It checks the operating system's metadata (os.path.getmtime) for the generated lookup tables. This prevents accidental overwriting or using stale data when resuming a session.

In [3]:
import os
import datetime

if 'INTEGRALS_PATH' in globals() and os.path.exists(INTEGRALS_PATH):
    mtime = os.path.getmtime(INTEGRALS_PATH)
    print(f"Last modification time of {os.path.basename(INTEGRALS_PATH)}: {datetime.datetime.fromtimestamp(mtime)}")
else:
    print(f"File not found or INTEGRALS_PATH not defined: {INTEGRALS_PATH if 'INTEGRALS_PATH' in globals() else 'Not Defined'}")

File not found or INTEGRALS_PATH not defined: ./AJ_Tables_g30_cutaware/aj_integrals_genus30_cutaware.pt


### **Cell 3: Geometry Consistency Check**

**Mathematical Context:**
Ensures that the geometric definition of the curve (the branch points $a_i$) in the current memory matches what is stored on disk.
*   The curve is defined by the set of points $\{a_1, \dots, a_{62}\}$.
*   If these points differ by even a small epsilon, the resulting integrals $I_k(z)$ will be mathematically invalid for the stored model.
*   We verify: $\max | a_{disk} - a_{mem} | < \epsilon$.

In [4]:
# @title Check/lock branch cuts against saved files
import os, numpy as np, torch

AUTO_ADOPT_SAVED_CUTS = False   

def pick_saved_path():
    cand = []
    if 'INTEGRALS_PATH' in globals() and os.path.exists(INTEGRALS_PATH): cand.append(INTEGRALS_PATH)
    if 'OMEGAS_PATH' in globals() and os.path.exists(OMEGAS_PATH): cand.append(OMEGAS_PATH)
    if not cand: return None
    cand.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    return cand[0]

def np_c128_from_list(lst):
    return np.array([complex(z) for z in lst], dtype=np.complex128)

if 'branch_pts_list' in globals(): current_pts_source = branch_pts_list
else: raise RuntimeError("Expected `branch_pts_list` from Cell 1.")

saved_path = pick_saved_path()

if saved_path is None:
    print("No saved ω/Ι file found — fresh run.")
else:
    pkg = torch.load(saved_path, map_location="cpu", weights_only=False)
    saved_genus = int(pkg.get("genus"))
    saved_grid_r = np.array(pkg.get("grid_r"))
    saved_grid_i = np.array(pkg.get("grid_i"))
    saved_raw_pts = pkg.get("branch_pts", [])
    if isinstance(saved_raw_pts, torch.Tensor): saved_raw_pts = saved_raw_pts.numpy()
    saved_pts = np.array(saved_raw_pts, dtype=np.complex128)
    saved_cuts = pkg.get("branch_cuts", None)

    curr_pts = np_c128_from_list(current_pts_source)

    print(f"Comparing to: {os.path.basename(saved_path)}")
    ok_genus = (saved_genus == genus)
    ok_grid  = np.allclose(saved_grid_r, grid_r) and np.allclose(saved_grid_i, grid_i)
    
    if saved_pts.size == curr_pts.size:
        saved_flat = saved_pts.flatten()
        curr_flat = curr_pts.flatten()
        ok_pts = np.allclose(saved_flat.real, curr_flat.real) and np.allclose(saved_flat.imag, curr_flat.imag)
    else:
        ok_pts = False

    print(f"  genus match : {ok_genus}")
    print(f"  grid match  : {ok_grid}")
    print(f"  branch pts  : {'MATCH' if ok_pts else 'MISMATCH'}")

    if ok_genus and ok_grid and ok_pts:
        print("\n✅ SAFE to resume.")
    else:
        print("\n❌ MISMATCH detected.")
        if AUTO_ADOPT_SAVED_CUTS and (saved_cuts is not None):
            branch_cuts = saved_cuts
            branch_pts_list = [a for ab in branch_cuts for a in ab]
            # UPDATE GPU TENSOR
            branch_pts_tensor = torch.tensor(branch_pts_list, device=DEVICE, dtype=DTYPE).reshape(1, 1, -1)
            print("→ Adopted saved cuts and updated GPU tensors.")

No saved ω/Ι file found — fresh run.



### **Cell 4: Build $\omega$ (Differentials)**

**Mathematical Context:**
This cell evaluates the holomorphic differentials on the lattice $\Lambda$.
1.  For each $k \in \{0, \dots, 29\}$, we compute the scalar field $\omega_k(z)$.
2.  **Vectorization:** Instead of iterating pixel-by-pixel, we process an entire row (Width $W$) as a single tensor operation on the GPU.
3.  The `omega_torch` function handles the complex log-math to prevent numerical instability.



In [5]:
# @title Build ω (differentials) for g=30 (GPU Accelerated)
import torch

# 1. Initialize
if os.path.exists(OMEGAS_PATH):
    pkg = torch.load(OMEGAS_PATH, map_location='cpu', weights_only=False)
    ensure_config_compatible(pkg)
    Om_plus = pkg["omega_plus"]
    progress = pkg.get("progress", {"iy_done": [0]*genus, "k_done": 0})
    print("Resuming ω from Drive.")
else:
    # Use standard complex64 for storage to save RAM/Disk, even if calc is high prec
    Om_plus = torch.zeros(genus, H, W, dtype=torch.complex64)
    progress = {"iy_done": [0]*genus, "k_done": 0}
    print("Starting fresh ω build.")

# Prepare Grid X on Device
# Determine float type based on complex DTYPE
real_dtype = torch.float64 if DTYPE == torch.complex128 else torch.float32
grid_r_tensor = torch.tensor(grid_r, device=DEVICE, dtype=real_dtype)

SAVE_EVERY_N_ROWS = 20 

t0 = time.time()
for k in range(progress["k_done"], genus):
    start_iy = int(progress["iy_done"][k])
    if start_iy >= H:
        print(f"ω[{k}] already complete.")
        progress["k_done"] = k+1
        continue

    print(f"Building ω[{k}] rows {start_iy}..{H-1}")
    rows_since_save = 0
    
    for iy in tqdm(range(start_iy, H), desc=f"omega[{k}]"):
        y_val = grid_i[iy]
        
        # 1. Create Complex Row Z
        # Broadcast scalar y_val to match grid_r
        Z_row = torch.complex(grid_r_tensor, torch.full_like(grid_r_tensor, y_val))
        
        # 2. Compute on GPU
        row_vals = omega_torch(k, Z_row)
        
        # 3. Store on CPU
        Om_plus[k, iy, :] = row_vals.cpu()
        
        progress["iy_done"][k] = iy + 1
        rows_since_save += 1

        if rows_since_save >= SAVE_EVERY_N_ROWS:
            payload = {**COMMON_META, "omega_plus": Om_plus, "progress": progress}
            atomic_torch_save(payload, OMEGAS_PATH)
            rows_since_save = 0

    progress["k_done"] = k+1
    payload = {**COMMON_META, "omega_plus": Om_plus, "progress": progress}
    atomic_torch_save(payload, OMEGAS_PATH)

t1 = time.time()
print(f"ω table saved | elapsed {t1 - t0:.1f}s")

Starting fresh ω build.
Building ω[0] rows 0..95


omega[0]: 100%|████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 119.33it/s]


Building ω[1] rows 0..95


omega[1]: 100%|███████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 1638.65it/s]


Building ω[2] rows 0..95


...

Building ω[28] rows 0..95


omega[28]: 100%|██████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 2080.57it/s]


Building ω[29] rows 0..95


omega[29]: 100%|██████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 2050.63it/s]

ω table saved | elapsed 2.3s



### **Cell 5: Build $I$ (Integrals)**

**Mathematical Context:**
This computes the Abel-Jacobi map $I_k(z) = \int_{P_0}^{z} \omega_k$.
1.  **Path Integration:** We approximate the integral using a **Riemann Sum** along the straight path from the base point $P_0$ to $z$.
2.  **Discretization:** The path is divided into `STEPS=1000` intervals.
3.  **Parallelism:** We calculate the integral for all $z$ in a specific row simultaneously.
    *   Tensor Shape: `(Steps, Width)`
    *   We evaluate $\omega$ at all steps and all horizontal points at once, sum over the steps, and multiply by $dt$.


In [6]:
# @title Build I (integrals) for g=30 (GPU Accelerated)
import torch

# 1. Initialize
if os.path.exists(INTEGRALS_PATH):
    pkg = torch.load(INTEGRALS_PATH, map_location='cpu', weights_only=False)
    ensure_config_compatible(pkg)
    I_plus = pkg["I_plus"]
    progress = pkg.get("progress", {"iy_done": [0]*genus, "k_done": 0})
    print("Resuming integrals from Drive.")
else:
    I_plus = torch.zeros(genus, H, W, dtype=torch.complex64)
    progress = {"iy_done": [0]*genus, "k_done": 0}
    print("Starting fresh integrals build.")

# Config
SAVE_EVERY_N_ROWS = 10 
STEPS_QUALITY = 1000 # Higher = more precise integration

# Prepare Grid
real_dtype = torch.float64 if DTYPE == torch.complex128 else torch.float32
grid_r_tensor = torch.tensor(grid_r, device=DEVICE, dtype=real_dtype)

t0 = time.time()
for k in range(progress["k_done"], genus):
    start_iy = int(progress["iy_done"][k])
    if start_iy >= H:
        print(f"I[{k}] already complete.")
        progress["k_done"] = k+1
        continue

    print(f"Building I[{k}] rows {start_iy}..{H-1}")
    rows_since_save = 0
    
    for iy in tqdm(range(start_iy, H), desc=f"I[{k}]   "):
        y_val = grid_i[iy]
        
        # 1. Create Complex Row Z
        Z_row = torch.complex(grid_r_tensor, torch.full_like(grid_r_tensor, y_val))
        
        # 2. Compute Integral on GPU
        # Uses the Riemann Sum integrator defined in Cell 1
        row_vals = integrate_row_torch(k, Z_row, base_point_complex, steps=STEPS_QUALITY)
        
        # 3. Store
        I_plus[k, iy, :] = row_vals.cpu()
        
        progress["iy_done"][k] = iy + 1
        rows_since_save += 1

        if rows_since_save >= SAVE_EVERY_N_ROWS:
            payload = {**COMMON_META, "I_plus": I_plus, "progress": progress}
            atomic_torch_save(payload, INTEGRALS_PATH)
            rows_since_save = 0

    progress["k_done"] = k+1
    payload = {**COMMON_META, "I_plus": I_plus, "progress": progress}
    atomic_torch_save(payload, INTEGRALS_PATH)

t1 = time.time()
print(f"I table saved | elapsed {t1 - t0:.1f}s")

Starting fresh integrals build.
Building I[0] rows 0..95


I[0]   : 100%|██████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:01<00:00, 85.59it/s]


Building I[1] rows 0..95


I[1]   : 100%|█████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 348.59it/s]


Building I[2] rows 0..95


...

Building I[28] rows 0..95


I[28]   : 100%|████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 444.31it/s]


Building I[29] rows 0..95


I[29]   : 100%|████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 428.23it/s]

I table saved | elapsed 7.8s
